In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image, ImageTk
import pandas as pd
from pathlib import Path
import yaml
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

class BirdClassifierApp:
    def __init__(self, model_path='models/best_model.pth'):
        # Загружаем модель
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model, self.class_names, self.transform = self.load_model(model_path)
        
        # Создаем GUI
        self.root = tk.Tk()
        self.root.title("Классификатор птиц")
        self.root.geometry("800x600")
        
        self.setup_ui()
        
    def load_model(self, model_path):
        """Загружает модель и информацию о классах"""
        # Загружаем информацию о классах
        with open('data/processed/class_info.yaml', 'r', encoding='utf-8') as f:
            class_info = yaml.safe_load(f)
        
        class_names = class_info['class_names']
        idx_to_class = class_info['idx_to_class']
        
        # Создаем модель
        model = models.mobilenet_v2()
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
        
        # Загружаем веса
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(self.device)
        model.eval()
        
        # Трансформации
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
        
        return model, class_names, transform
    
    def setup_ui(self):
        """Настраивает интерфейс"""
        # Заголовок
        title_label = tk.Label(
            self.root, 
            text="Классификатор птиц", 
            font=("Arial", 20, "bold"),
            fg="#2c3e50"
        )
        title_label.pack(pady=20)
        
        # Фрейм для загрузки изображения
        upload_frame = tk.Frame(self.root)
        upload_frame.pack(pady=10)
        
        # Кнопка загрузки
        self.upload_btn = tk.Button(
            upload_frame,
            text="Загрузить изображение",
            command=self.load_image,
            font=("Arial", 12),
            bg="#3498db",
            fg="white",
            padx=20,
            pady=10,
            cursor="hand2"
        )
        self.upload_btn.pack(side=tk.LEFT, padx=10)
        
        # Кнопка классификации
        self.classify_btn = tk.Button(
            upload_frame,
            text="Классифицировать",
            command=self.classify_image,
            font=("Arial", 12),
            bg="#2ecc71",
            fg="white",
            padx=20,
            pady=10,
            state=tk.DISABLED,
            cursor="hand2"
        )
        self.classify_btn.pack(side=tk.LEFT, padx=10)
        
        # Кнопка сброса
        self.reset_btn = tk.Button(
            upload_frame,
            text="Сброс",
            command=self.reset_app,
            font=("Arial", 12),
            bg="#e74c3c",
            fg="white",
            padx=20,
            pady=10,
            cursor="hand2"
        )
        self.reset_btn.pack(side=tk.LEFT, padx=10)
        
        # Фрейм для изображения и результатов
        content_frame = tk.Frame(self.root)
        content_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Левая часть - изображение
        left_frame = tk.Frame(content_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.image_label = tk.Label(
            left_frame,
            text="Изображение не загружено",
            font=("Arial", 12),
            fg="#7f8c8d",
            relief=tk.SUNKEN,
            width=40,
            height=20
        )
        self.image_label.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Правая часть - результаты
        right_frame = tk.Frame(content_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Результат классификации
        result_frame = tk.LabelFrame(right_frame, text="Результат", font=("Arial", 14, "bold"))
        result_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.result_text = tk.Text(
            result_frame,
            height=5,
            width=30,
            font=("Arial", 11),
            state=tk.DISABLED
        )
        self.result_text.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Вероятности
        prob_frame = tk.LabelFrame(right_frame, text="Вероятности", font=("Arial", 14, "bold"))
        prob_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.prob_canvas = tk.Canvas(prob_frame, bg="white")
        self.prob_canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Статус бар
        self.status_bar = tk.Label(
            self.root,
            text="Готов к работе. Загрузите изображение птицы.",
            relief=tk.SUNKEN,
            anchor=tk.W,
            font=("Arial", 10)
        )
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Переменные
        self.current_image = None
        self.image_path = None
        
    def load_image(self):
        """Загружает изображение"""
        file_path = filedialog.askopenfilename(
            title="Выберите изображение",
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.bmp"),
                ("All files", "*.*")
            ]
        )
        
        if file_path:
            try:
                # Загружаем и отображаем изображение
                image = Image.open(file_path)
                image.thumbnail((400, 400))  # Уменьшаем для отображения
                
                # Конвертируем для Tkinter
                photo = ImageTk.PhotoImage(image)
                self.image_label.config(image=photo, text="")
                self.image_label.image = photo  # Сохраняем ссылку
                
                self.current_image = Image.open(file_path).convert('RGB')
                self.image_path = file_path
                
                # Активируем кнопку классификации
                self.classify_btn.config(state=tk.NORMAL)
                
                self.status_bar.config(
                    text=f"Загружено: {Path(file_path).name}",
                    fg="#27ae60"
                )
                
            except Exception as e:
                messagebox.showerror("Ошибка", f"Не удалось загрузить изображение: {e}")
                self.status_bar.config(text="Ошибка загрузки", fg="#c0392b")
    
    def classify_image(self):
        """Классифицирует изображение"""
        if self.current_image is None:
            messagebox.showwarning("Внимание", "Сначала загрузите изображение!")
            return
        
        try:
            # Подготовка изображения
            image_tensor = self.transform(self.current_image).unsqueeze(0)
            image_tensor = image_tensor.to(self.device)
            
            # Предсказание
            with torch.no_grad():
                outputs = self.model(image_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                probs, indices = torch.topk(probabilities, k=len(self.class_names))
            
            # Преобразуем в numpy
            probs = probs.cpu().numpy()[0]
            indices = indices.cpu().numpy()[0]
            
            # Находим лучший класс
            best_idx = indices[0]
            best_prob = probs[0]
            best_class = self.class_names[best_idx]
            
            # Показываем результат
            self.show_results(best_class, best_prob, indices, probs)
            
            # Обновляем статус
            self.status_bar.config(
                text=f"Классифицировано как: {best_class} ({(best_prob*100):.1f}%)",
                fg="#2980b9"
            )
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка при классификации: {e}")
            self.status_bar.config(text="Ошибка классификации", fg="#c0392b")
    
    def show_results(self, best_class, best_prob, indices, probs):
        """Отображает результаты классификации"""
        # Основной результат
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        result_text = f"Результат:\n"
        result_text += f"Класс: {best_class}\n"
        result_text += f"Вероятность: {(best_prob*100):.1f}%\n\n"
        result_text += f"Топ-3 предсказания:\n"
        
        for i in range(3):
            class_name = self.class_names[indices[i]]
            prob = probs[i] * 100
            result_text += f"{i+1}. {class_name}: {prob:.1f}%\n"
        
        self.result_text.insert(1.0, result_text)
        self.result_text.config(state=tk.DISABLED)
        
        # Визуализация вероятностей
        self.draw_probabilities(indices[:5], probs[:5])
    
    def draw_probabilities(self, indices, probs):
        """Рисует график вероятностей"""
        # Очищаем canvas
        self.prob_canvas.delete("all")
        
        # Данные для графика
        classes = [self.class_names[i] for i in indices]
        percentages = [p * 100 for p in probs]
        
        # Параметры
        canvas_width = self.prob_canvas.winfo_width()
        canvas_height = self.prob_canvas.winfo_height()
        
        if canvas_width < 10:  # Если canvas еще не отрисован
            canvas_width = 300
            canvas_height = 200
        
        # Настройки
        bar_width = 20
        max_height = canvas_height - 80
        x_spacing = 50
        x_start = 50
        
        # Находим максимум для масштабирования
        max_prob = max(percentages) if percentages else 1
        
        # Рисуем столбцы
        for i, (cls, prob) in enumerate(zip(classes, percentages)):
            x = x_start + i * x_spacing
            bar_height = (prob / max_prob) * max_height
            
            # Координаты столбца
            y1 = canvas_height - 30 - bar_height
            y2 = canvas_height - 30
            
            # Цвет (градиент от зеленого к красному)
            color = self.get_color_gradient(prob/100)
            
            # Рисуем столбец
            self.prob_canvas.create_rectangle(
                x, y1, x + bar_width, y2,
                fill=color,
                outline="black"
            )
            
            # Подпись вероятности
            self.prob_canvas.create_text(
                x + bar_width/2, y1 - 10,
                text=f"{prob:.1f}%",
                font=("Arial", 9)
            )
            
            # Подпись класса
            self.prob_canvas.create_text(
                x + bar_width/2, y2 + 15,
                text=cls[:8] + ("..." if len(cls) > 8 else ""),
                font=("Arial", 9),
                angle=45
            )
        
        # Ось Y
        self.prob_canvas.create_line(
            30, 30, 30, canvas_height - 30,
            width=2
        )
        
        # Подписи оси Y
        for pct in [0, 25, 50, 75, 100]:
            y = canvas_height - 30 - (pct/100) * max_height
            self.prob_canvas.create_text(
                25, y,
                text=f"{pct}%",
                font=("Arial", 8),
                anchor=tk.E
            )
    
    def get_color_gradient(self, value):
        """Возвращает цвет в зависимости от значения (0-1)"""
        # От красного (0) к желтому (0.5) к зеленому (1)
        if value < 0.5:
            # Красный -> желтый
            r = 255
            g = int(255 * (value * 2))
            b = 0
        else:
            # Желтый -> зеленый
            r = int(255 * (2 - value * 2))
            g = 255
            b = 0
        
        return f'#{r:02x}{g:02x}{b:02x}'
    
    def reset_app(self):
        """Сбрасывает приложение"""
        self.current_image = None
        self.image_path = None
        self.image_label.config(image=None, text="Изображение не загружено")
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        self.result_text.config(state=tk.DISABLED)
        self.prob_canvas.delete("all")
        self.classify_btn.config(state=tk.DISABLED)
        self.status_bar.config(text="Готов к работе. Загрузите изображение птицы.", fg="black")
    
    def run(self):
        """Запускает приложение"""
        self.root.mainloop()

def main():
    app = BirdClassifierApp()
    app.run()

if __name__ == "__main__":
    main()